In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [45]:
import math
from creme import datasets
from creme import linear_model
from creme import metrics
from creme import model_selection
from creme import preprocessing
from creme import stats
from creme import stream
from sklearn import datasets

X_y = stream.iter_sklearn_dataset(
    datasets.load_boston(),
    shuffle=True,
    random_state=42
)

model = (
    preprocessing.StandardScaler() |
    linear_model.LinearRegression(intercept_lr=.1)
)

metric = metrics.MAE()

n = 0
n_in = 0
se = stats.RollingSum(1000)

for x, y in X_y:
    y_pred = model.predict_one(x)
    
    if metric.n > 2:
        stdev = math.sqrt(1 / (metric.n - 2) * se.get())
        interval = 1.96 * stdev
        lower, upper = y_pred - interval, y_pred + interval
        
        n_in += lower < y < upper
        n += 1
        
    se.update((y - y_pred) ** 2)
    metric.update(y, y_pred)
    model.fit_one(x, y)
    
metric, n_in / n

(MAE: 4.038378, 0.9642147117296223)

In [43]:
metric.n

506.0

In [13]:
stdev

0.07329962081200625